# Pre-process wflow_sbm forcing for CAMELS-GB
## CEH-GEAR: pr, CHESS-PE: pet, CHESS-met: tas

In [1]:
from glob import glob
from pathlib import Path

import iris
import xarray as xr
import pandas as pd
from esmvalcore.preprocessor import regrid

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Set Paths

In [2]:
# Snellius paths
ROOT = '/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/'
AUXDATA = Path(f"{ROOT}/aux_data")
FORCING = Path(f'{ROOT}/forcing/')
MODELS = Path(f'{ROOT}/wflow/data/')
NOTEBOOKS = Path(f'{ROOT}/notebooks')

# Config

In [3]:
# Papermill Parameters
basin_id = '24004'


In [4]:
#Time Period
start_year = "2000"
end_year = "2017"

# Forcing pre-process function

In [5]:
# Prepare forcing function
def prep_forcing(basin_id):
    
    # Create output list
    output_cubes = []
    
    # Set basin directory
    BASINDIR = f'{MODELS}/{basin_id}/'
       
    # Open netCDF file as an example grid from the model directory
    cube_example = iris.load(f'{BASINDIR}/staticmaps.nc')[1]

    # Rename Coords
    cube_example.coord('x').rename('longitude')
    cube_example.coord('y').rename('latitude')

    # Guess bounds
    cube_example.coord('latitude').guess_bounds()
    cube_example.coord('longitude').guess_bounds()

    # Loop forcing variables
    for variable in ['pr','tas','pet']:
 
        # Load forcing file
        cube_forcing = iris.load(glob(f'{FORCING}/*{variable}*')[0])[0]

        # Rename Coords
        cube_forcing.coord('lon').rename('longitude')
        cube_forcing.coord('lat').rename('latitude')

        # Guess bounds
        cube_forcing.coord('latitude').guess_bounds()
        cube_forcing.coord('longitude').guess_bounds()

        # Regrid forcing file to example grid using conservative method
        cube_out = regrid(cube_forcing, cube_example, scheme='area_weighted')

        # Append to cube list
        output_cubes.append(cube_out)

    # Combine output cubes
    cubes = iris.cube.CubeList(output_cubes)
    
    # Output filename
    output_fname = f'{BASINDIR}/ceh-gear_chess_camels-gb_{basin_id}_2000_2017.nc'
    
    # Remove existing file
    if output_fname:
        OUTPUT = Path(output_fname)
        OUTPUT.unlink(output_fname)
    
    # Save output cubes
    iris.save(cubes, output_fname)

    return print(f'{basin_id} finished')

# Run Function

In [6]:
prep_forcing(basin_id)

24004 finished
